In [2]:
import pandas as pd
import requests
import folium
import geopandas as gpd
import shapely
from shapely.geometry import Polygon

In [3]:
PICKLE_PATH = "/Users/abby/Desktop/RAFI/pickled-beef.pickle"
DATA = "/Users/abby/Desktop/RAFI/fsis-processors-with-location.csv"

In [4]:
full_df = pd.read_csv(DATA)
full_df.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [5]:
beef = full_df[full_df['Animals Processed'].str.contains('Beef')]
len(beef)

742

In [8]:
large_beef = beef.loc[beef.Size == "Large"]
print("This dataframe is", len(large_beef), "cells long")
large_beef.head()

This dataframe is 41 cells long


,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [9]:
MAPBOX_TOKEN = "pk.eyJ1IjoidG9kZG5pZWYiLCJhIjoiY2xncGpzbmhhMTBwdzNnbXJjNWlzaTY2aCJ9.UhUELBA2iNIDsTN9YESsIw"
ENDPOINT = "https://api.mapbox.com/isochrone/v1/mapbox/driving/"
DRIVING_DISTANCE = str(int(60 * 1609.34))

In [10]:
isochrones = []
for index, row in large_beef.iterrows():
    lat = str(row['latitude'])
    lng = str(row['longitude'])

    # add driving radius isochrone to map
    url = ENDPOINT + lng + "," + lat + "?" + "contours_meters=" + DRIVING_DISTANCE + "&access_token=" + MAPBOX_TOKEN
    response = requests.get(url)
    isochrone = Polygon(response.json()['features'][0]['geometry']['coordinates'])
    isochrones.append(isochrone)

In [12]:
df_large_beef = large_beef.copy()
df_large_beef['Isochrone'] = isochrones
df_large_beef.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Isochrone
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559,"POLYGON ((-112.138559 34.159866, -112.142615 3..."
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950,"POLYGON ((-119.63395 37.448916, -119.634798 37..."
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222,"POLYGON ((-119.753222 37.089778, -119.756222 3..."
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553,"POLYGON ((-119.695553 37.26998, -119.704553 37..."
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786,"POLYGON ((-115.449786 33.792326, -115.450997 3..."


In [13]:
df_large_beef.to_pickle(PICKLE_PATH)